In [9]:
from src.env.briscola.TwoPlayerBriscola import TwoPlayerBriscola
from src.env.briscola.BriscolaState import BriscolaState

env = TwoPlayerBriscola()

print(env.state.deck)

print(env.state.player_points, env.state.player_order, env.state.player_cards)
env.step(1)
env.step(2)
print(env.state.player_points, env.state.player_order, env.state.player_cards)

[8B, 5B, 10D, 6C, 5S, 2C, 1C, 9D, 6S, 7B, 3S, 4B, 1S, 10C, 9C, 9B, 10B, 7S, 7D, 1D, 2S, 2D, 10S, 8D, 3D, 4C, 3C, 2B, 5C, 5D, 1B, 4S, 8S, 6D]
[0, 0] [1, 0] [[7C, 8C, 9S], [6B, 3B, 4D]]
3B 9S 3
0 13
[0, 13] [1, 0] [[7C, 8C, 8S], [6B, 4D, 6D]]


In [8]:
from src.env.briscola.Card import Card

p1_card, p2_card = Card(34), Card(31)

env.get_score(p1_card, p2_card)

(0, 0)

In [24]:
from src.env.briscola.utils import get_rank

Card(3).get_priority()

2